In [15]:
# Import required modules
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Create variables for scraped data
OFF_TEAM = []
DEF_TEAM = []
FPTS = []
WEEK = []
pos = 'DEF'
pos_dict = {'QB':1,'RB':2,'WR':3,'TE':4,'K':7,'DEF':8}
pos_key = pos_dict[pos]
week_num = 7

for i in range(week_num):

    url = "https://fantasy.nfl.com/research/pointsagainst?position=" + str(pos_key) + "&statCategory=pointsAgainst&statSeason=2021&statType=weekPointsAgainst&statWeek=" + str(i+1)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    time.sleep(1)

    # Create an object of the first object that is class=data compact
    table = soup.find('tbody')

    # Find all the <tr> tag pairs, skip the first one, then for each.
    for row in table.find_all('tr'):
        
        # Create a variable of all the <td> tag pairs in each <tr> tag pair,
        #col = row.find('td').text

        DEF_TEAM.append(row.find_all('td')[0].text)
        OFF_TEAM.append(row.find_all('td')[1].text)
        FPTS.append(row.find_all('td')[2].text)
        WEEK.append(i+1)

    # Create a variable of the value of the columns
col = {'OFF_TEAM': OFF_TEAM,
       'DEF_TEAM': DEF_TEAM,
       'FPTS': FPTS,
       'WEEK': WEEK,
       }

    # Create a dataframe from the columns variable
df = pd.DataFrame(col)
df['POSITION'] = df['DEF_TEAM'].apply(lambda x: x.split('vs ')[1])
df['DEF_TEAM'] = df['DEF_TEAM'].apply(lambda x: x.split('vs ')[0]).str.replace(" Defense", "")
df['OFF_TEAM'] = df['OFF_TEAM'].apply(lambda x: x.replace("@", ""))
#df['DEF_TEAM'] = df['DEF_TEAM'].apply(lambda x: x.replace("WAS", "WSH"))
#df['DEF_TEAM'] = df['DEF_TEAM'].apply(lambda x: x.replace("LA", "LAR"))
df['OFF_TEAM'] = df['OFF_TEAM'].apply(lambda x: 'WSH' if x=='WAS' else x)
df['OFF_TEAM'] = df['OFF_TEAM'].apply(lambda x: 'LAR' if x=='LA' else x)
#df = df[df.TEAM.str.contains("vs ")]
#df["OFF_TEAM"], df["POSITION"], df["VS"] , df["DEF_TEAM"], df["JUNK"] = zip(*df["TEAM"].str.split().tolist())
df = df[["WEEK", "POSITION", "OFF_TEAM", "DEF_TEAM" , "FPTS"]]
df['FPTS'] = df['FPTS'].astype(float).astype(int)
df.to_csv("FF_Data", index=False)
df

,WEEK,POSITION,OFF_TEAM,DEF_TEAM,FPTS
0,1,DEF,ARI,Tennessee Titans,16
1,1,DEF,NO,Green Bay Packers,15
2,1,DEF,PIT,Buffalo Bills,12
3,1,DEF,PHI,Atlanta Falcons,10
4,1,DEF,SF,Detroit Lions,10
...,...,...,...,...,...
219,7,DEF,Bye,Jacksonville Jaguars,0
220,7,DEF,DET,Los Angeles Rams,0
221,7,DEF,BAL,Cincinnati Bengals,-1
222,7,DEF,CHI,Tampa Bay Buccaneers,-2


In [16]:
import pandas as pd
df = pd.read_csv("FF_Data")
df = df[df['OFF_TEAM'] != 'Bye']
df2 = pd.read_csv("Team Index.csv")
schedule = pd.read_csv("Team Schedule.csv")
team_index = df2[['Abb','Team']]

melted_schedule = pd.melt(schedule, var_name = 'Week', value_name='Team_ID')
melted_schedule = melted_schedule[melted_schedule['Week'] != 'TEAM']
home_teams = pd.merge(melted_schedule, team_index, left_on=['Team_ID'], right_on=['Abb'])
home_teams['Week'] = home_teams['Week'].astype(int)

data = pd.merge(df, df2, how='inner', left_on = 'OFF_TEAM', right_on = 'Abb')
data = data.rename(columns={'Index': 'OFF_Index'})
data = pd.merge(data, df2, how='inner', left_on = 'DEF_TEAM', right_on = 'FullName')
data = data.rename(columns={'Index': 'DEF_Index'})
data = pd.merge(data, data[['OFF_Index','FPTS', 'WEEK']], how='inner', left_on = ['DEF_Index', 'WEEK'], right_on = ['OFF_Index', 'WEEK'])
data = pd.merge(data, home_teams, how='inner', left_on=['Abb_x','WEEK'], right_on = ['Abb','Week'])
data = data[['DEF_Index', 'OFF_Index_x', 'FPTS_y', 'FPTS_x']]
data.columns = ['HOME', 'AWAY', 'HOME_FPTS', 'AWAY_FPTS']
games = data.values.tolist()

import numpy as np
def buildGamesMatrix2(games, num_teams):
    l = len(games)
    M = np.zeros([l*2, num_teams*2+1])
    for i in range(l):
        g = games[i]
        M[i, g[0]] += 1
        M[i, g[1]+num_teams] += -1
        M[i, num_teams*2] += 1
        M[l+i, g[1]] += 1
        M[l+i, g[0]+num_teams] += -1
    return M

M2 = buildGamesMatrix2(games,32)
#print(M2)

def buildOutcomes2(games, n):
    l = len(games)
    E = np.zeros([l*2])
    for i in range(l):
        g = games[i]
        E[i] += g[2]
        E[l+i] += g[3]
    return E

E2 = buildOutcomes2(games, 32)
#print(E2)

from sklearn import linear_model
clf = linear_model.Ridge(fit_intercept=False, alpha=0.1)
#clf = linear_model.LinearRegression(fit_intercept=True)

clf.fit(M2,E2)

R = clf.coef_

results_df = pd.DataFrame()
schedule2 = pd.read_csv("Team Schedule.csv")
WEEK = week_num+1

for i in range(32):
    temp = pd.DataFrame({'Team': i, 'For': R[i], 'Against': R[i+32]}, index=[0])
    results_df = pd.concat([results_df, temp])

results_df = pd.merge(results_df, df2, how='inner', left_on = 'Team', right_on = 'Index')
results_df = pd.merge(results_df, schedule2, how='inner', left_on = 'Abb', right_on = 'TEAM')
results_df.rename(columns={str(WEEK): 'OPP'}, inplace=True)
results_df['OPP'] = results_df['OPP'].replace({'@':''}, regex=True)
results_df = results_df[['TEAM', 'Index', 'For', 'Against', 'OPP']]
results_df = pd.merge(results_df, results_df, left_on = 'TEAM', right_on = 'OPP')
results_df['Projected'] = results_df['For_x'] - results_df['Against_y']
results_df['Position'] = pos
results_df = results_df[['Position', 'TEAM_x', 'For_x', 'OPP_x', 'Against_y', 'Projected']]
results_df.sort_values(by='Against_y', ascending=[1])

#save and/or append results to csv
#results_df.to_csv('FF_Results.csv', mode='a', header=False, index = False)
#results_df.to_csv('FF_Results.csv', header=True, index = False)


,Position,TEAM_x,For_x,OPP_x,Against_y,Projected
2,DEF,CIN,2.380315,NYJ,-8.785075,11.165389
0,DEF,SF,2.312918,CHI,-6.417266,8.730185
21,DEF,CAR,2.638861,ATL,-6.315851,8.954712
15,DEF,WSH,1.400651,DEN,-6.110284,7.510934
25,DEF,SEA,2.755664,JAX,-5.657086,8.412750
16,DEF,NYG,2.133412,KC,-5.108858,7.242270
23,DEF,LAR,4.358690,HOU,-5.030228,9.388918
14,DEF,ATL,-0.780961,CAR,-4.857563,4.076602
8,DEF,LAC,2.615291,NE,-4.419503,7.034794
3,DEF,BUF,8.419497,MIA,-4.223893,12.643390
